In [1]:
# Import libraries
# !pip install lime
import os
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import random
# import lime
from lime import lime_tabular
import shap

import json
from typing import Union
from datetime import datetime as dt
import numpy as np


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# from dask.distributed import Client

# # client = Client( n_workers=2, )
# client = Client(n_workers=4, threads_per_worker=1, memory_limit='6GB',dashboard_address=':8087', timeout="600s")

# client


In [3]:
# import dask.dataframe as dd

In [4]:
PROCESSED_DATA = "./fold_data/processed_final_data.csv"
SAMPLE_ID =  "./fold_data/sampled_data_project_ids.csv"

In [5]:
raw_data = pd.read_csv(PROCESSED_DATA)
sampled_raw_data = pd.read_csv(SAMPLE_ID)

In [6]:
#RuntimeError: No visualization engine detected, please install graphviz or ipycytoscape
# !pip install graphviz

In [7]:
# Dask constructs logic called task graph of your computationimmediately them only when necessary. this graph can be visualized as below
# raw_data.visualize()

In [8]:
# sampled_raw_data.tail()

In [9]:
filtered_data = raw_data[raw_data["Project ID"].isin(sampled_raw_data["Project ID"])]

In [10]:
filtered_data.shape

(300000, 28)

In [11]:
filtered_data.head()

,Unnamed: 0,Project ID,Project Posted Date,Project Type,Project Subject Category Tree,Project Cost,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,School Metro Type,...,Label,Teacher Success Rate,Teacher Success Rate Imputed,School City Success Rate,School City Success Rate Imputed,School Success Rate,School Success Rate Imputed,School County Success Rate,School County Success Rate Imputed,Project Count in State
0,0,fc71f1392dc1804262ca29696d5a1c6c,2014-01-03,Teacher-Led,"Math & Science, Literacy & Language",326.12,"Environmental Science, Literacy",Grades 9-12,Books,urban,...,0,0.0,1,0.000000,1,0.000000,1,0.000000,1,0
1,1,7bb325d4691141fc001854e87e6506db,2016-09-08,Teacher-Led,Math & Science,297.95,"Applied Sciences, Mathematics",Grades PreK-2,Supplies,urban,...,0,0.0,1,0.803030,0,0.666667,0,0.790951,0,2851
3,5,b200ed5bf0d9e113d3d50c26e3b98bc5,2016-11-26,Teacher-Led,Literacy & Language,305.26,"Literacy, Literature & Writing",Grades PreK-2,Books,rural,...,0,0.0,1,0.714286,0,1.000000,0,1.000000,0,1051
4,6,325a84427633fac794bca9ae7d3915f6,2015-02-24,Teacher-Led,Applied Learning,487.86,Parent Involvement,Grades PreK-2,Supplies,suburban,...,0,1.0,0,1.000000,0,1.000000,0,0.688679,0,2093
6,8,909083bd12a631a76cf828ac454cf0a0,2015-08-21,Teacher-Led,Literacy & Language,295.45,"ESL, Literacy",Grades 9-12,Technology,suburban,...,0,0.0,1,0.800000,0,0.000000,1,0.763473,0,1244


In [12]:

os.chdir("code")
import os
import feature_engineer as fe 
import data_processor as dp
import config
from helper import (
    save_model,
    load_model,
    create_dirs,
    create_classification_models,
    create_logistic_regression_parameters,
    create_random_forest_parameters,
    create_xgb_classifier_parameters,
    log_intermediate_output_to_file,
    filter_dataset_by_date)
import temporal_features as tmpf
import config

In [13]:
print("Loading already processed data")
# data = dp.load_data_to_df(path=data_file_path)
data = dp.set_data_types_to_datetime(filtered_data, ["Project Posted Date"])
data = filter_dataset_by_date(data)


# data_folds = fe.split_data_folds(data_1)


Loading already processed data



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
# from dask_ml.preprocessing import DummyEncoder
data = data.drop(columns=["School Name"])
data_1 = dp.encode_data(data, config.CATEGORICAL_COLS)
# ddf= dd.from_pandas(data, npartitions=8)
# print("encoded_data.shape = ", data_1.shape)


In [15]:
# from dask_ml.preprocessing import Categorizer, DummyEncoder
# from sklearn.pipeline import make_pipeline

In [16]:
# ddf = ddf.categorize(config.CATEGORICAL_COLS)

# # Then generate an encoder. 
# encoder = DummyEncoder() 

# # Fit the encoder 
# ddf_enc = encoder.fit_transform(ddf[config.CATEGORICAL_COLS])  

In [17]:
# ddf_columns = [col for col in ddf.columns if col not in config.CATEGORICAL_COLS ]

In [18]:
# ddf = dd.concat([ddf[ddf_columns], ddf_enc], axis=1)

In [19]:
# ddf.visualize()

In [20]:
# ddf_computed = ddf.compute()

In [21]:
# ddf_computed.head()

In [22]:
data_folds = fe.split_data_folds(data_1)

-----
train    2016-01-07 - 2016-07-05
test     2016-11-02 - 2016-12-02
(123763, 19217)
(47625, 19216)
(47625, 19215)
(47625, 1)
(16667, 19217)
(10321, 19216)
(10321, 19215)
(10321, 19215)
Training set shape =  (47625, 19215)
Testing set shape =  (10321, 19215)
-----
-----
train    2015-09-09 - 2016-03-07
test     2016-07-05 - 2016-08-04
(163010, 19217)
(55144, 19216)
(55144, 19215)
(55144, 1)
(75910, 19217)
(9789, 19216)
(9789, 19215)
(9789, 19215)
Training set shape =  (55144, 19215)
Testing set shape =  (9789, 19215)
-----
-----
train    2015-05-12 - 2015-11-08
test     2016-03-07 - 2016-04-06
(192343, 19217)
(51845, 19216)
(51845, 19215)
(51845, 1)
(107527, 19217)
(12220, 19216)
(12220, 19215)
(12220, 19215)
Training set shape =  (51845, 19215)
Testing set shape =  (12220, 19215)
-----
-----
train    2015-01-12 - 2015-07-11
test     2015-11-08 - 2015-12-08
(215977, 19217)
(32090, 19216)
(32090, 19215)
(32090, 1)
(139968, 19217)
(9634, 19216)
(9634, 19215)
(9634, 19215)
Training set

In [23]:
# # Initiate timing variables
# def get_positive_percentage(y_train , y_test):
#     train_pos = y_train["Label"].value_counts()[1] / len(y_train["Label"])
#     test_pos = y_test["Label"].value_counts()[1] / len(y_test["Label"])

#     return train_pos, test_pos

# import dask.dataframe as dd
# import dask.delayed as delayed

# from datetime import timedelta
# max_t = pd.Timestamp(config.MAX_TIME)
# min_t = pd.Timestamp(config.MIN_TIME)
# shift_period = timedelta(days=config.LEAK_OFFSET)   # 4 months
# fold_period = timedelta(days=config.WINDOW)    # 15 months

# t_current = max_t
# folds = 1

# folded_dataset = []
# # print(data.columns)

# data = dd.from_pandas(ddf.compute(),npartitions=2)

# while t_current > min_t + fold_period:
#     start_date = t_current - fold_period

#     delayed_result = delayed(dp.split_temporal_train_test_data(
#         data=data,
#         start_date=start_date)
#     )
    
#     x_train, y_train, x_test, y_test = delayed_result.compute()
    

#     # Count the positive labeled percentage in the training set and the test set
#     train_pos_perc, test_pos_perc = get_positive_percentage(
#         y_train, y_test)

#     fold_dataset = {
#         "fold_no": folds,
#         "start_date": start_date,
#         "x_train": x_train,
#         "y_train": y_train,
#         "x_test": x_test,
#         "y_test": y_test,
#         "train_pos_perc": train_pos_perc,
#         "test_pos_perc": test_pos_perc
#     }
#     folded_dataset.append(fold_dataset)

#     train_end = start_date + timedelta(config.TRAIN_SIZE)
#     test_start = train_end + timedelta(config.LEAK_OFFSET)
#     test_end = test_start + timedelta(config.TEST_SIZE)

#     fold_info = {
#         'fold_number': folds,
#         'timeline': {
#             'train_start': str(start_date)[:10],
#             'train_end': str(train_end)[:10],
#             'test_start': str(test_start)[:10],
#             'test_end': str(test_end)[:10]
#         },
#         'shape': {
#             'training_shape': str(x_train.shape),
#             'test_shape': str(x_test.shape),
#         },
#         'data_distribution': {
#             'train_positive_ratio': train_pos_perc,
#             'test_positive_ratio': test_pos_perc
#         }
#     }
#     file_name = f"Fold {folds} - {str(start_date)[:10]}.json"
#     dp.save_json(fold_info, config.INFO_DEST+file_name)

#     # Combine x_train and y_train into a single DataFrame
#     train_df_tmp = dd.concat([x_train, y_train], axis=1).compute()
#     train_df = dd.concat(
#      [data.loc[train_df_tmp.index]["Project ID"], train_df_tmp], axis=1)

#     # Combine x_test, y_test, and predicted_probabilities into a single DataFrame
#     test_df_tmp = dd.concat([x_test, y_test], axis=1).compute()
#     test_df = dd.concat(
#      [data.loc[test_df_tmp.index]["Project ID"], test_df_tmp], axis=1)

#     art_path = config.ARTIFACTS_PATH
#     train_df.to_csv(
#     art_path + f'train_fold_{folds}_{str(start_date)[:10]}.csv', single_file=True
#     )
#     test_df.to_csv(
#     art_path + f'test_fold_{folds}_{str(start_date)[:10]}.csv', single_file=True
#     )
#     log_intermediate_output_to_file(
#     config.INFO_DEST, config.PROGRAM_LOG_FILE, f'Fold {folds} is done.')
data_folds[0].keys()

dict_keys(['fold_no', 'start_date', 'x_train', 'y_train', 'x_test', 'y_test', 'train_pos_perc', 'test_pos_perc'])

Define the necessary paths to obtain data sources after running the main pipeline

In [24]:
root_test = "/home_2/university/fyp/cepdnaclk_2/e17-4yp-using-machine-learning-in-high-stake-settings/fold_data/random_forest_t_200_md_4"

In [25]:
# root_train = "/storage/scratch/e17-4yp-xai/cepdnaclk/e17-4yp-using-machine-learning-in-high-stake-settings/code/model_outputs/info"

In [26]:
# # train_files = sorted(['Fold 1 - 2016-01-07.json', 'Fold 4 - 2015-01-12.json',
# 'Fold 2 - 2015-09-09.json', 'Fold 5 - 2014-09-14.json',
# 'Fold 3 - 2015-05-12.json',  'Fold 6 - 2014-05-17.json'], reverse=True)

In [27]:
# # test_files = sorted(["test_prediction_fold_1_2016-01-07.csv",  "test_prediction_fold_4_2015-01-12.csv",
# "test_prediction_fold_2_2015-09-09.csv",  "test_prediction_fold_5_2014-09-14.csv",
# "test_prediction_fold_3_2015-05-12.csv",  "test_prediction_fold_6_2014-05-17.csv"], reverse=True)

In [28]:
models = sorted(["random_forest_t_200_md_4_fold_1_2016-01-07.pkl",
"random_forest_t_200_md_4_fold_2_2015-09-09.pkl",
"random_forest_t_200_md_4_fold_3_2015-05-12.pkl",
"random_forest_t_200_md_4_fold_4_2015-01-12.pkl",
"random_forest_t_200_md_4_fold_5_2014-09-14.pkl",
"random_forest_t_200_md_4_fold_6_2014-05-17.pkl"], reverse=True)

In [29]:
# train_files = sorted(['train_fold_1_2017-01-06.csv', 'train_fold_2_2016-09-08.csv', 'train_fold_3_2016-05-11.csv', 'train_fold_4_2016-01-12.csv', 'train_fold_5_2015-09-14.csv' , 'train_fold_6_2015-05-17.csv'], reverse=True)
# test_files = [ 'test_fold_6_2015-05-17.csv' , 'test_fold_5_2015-09-14.csv', 'test_fold_4_2016-01-12.csv', 'test_fold_3_2016-05-11.csv',  'test_fold_2_2016-09-08.csv', 'test_fold_1_2017-01-06.csv']


test_pred = sorted(["test_prediction_fold_1_2016-01-07.csv" , "test_prediction_fold_4_2015-01-12.csv",
"test_prediction_fold_2_2015-09-09.csv",  "test_prediction_fold_5_2014-09-14.csv"
,"test_prediction_fold_3_2015-05-12.csv",  "test_prediction_fold_6_2014-05-17.csv" ], reverse = True)

models = sorted(["random_forest_t_200_md_4_fold_1_2016-01-07.pkl",
"random_forest_t_200_md_4_fold_2_2015-09-09.pkl",
"random_forest_t_200_md_4_fold_3_2015-05-12.pkl",
"random_forest_t_200_md_4_fold_4_2015-01-12.pkl",
"random_forest_t_200_md_4_fold_5_2014-09-14.pkl",
"random_forest_t_200_md_4_fold_6_2014-05-17.pkl" ], reverse = True)

In [30]:
#### SET PATH
# ROOT = "/storage/scratch/e17-4yp-xai/Documents/e17-4yp-using-machine-learning-in-high-stake-settings/code/"
ROOT = "./"
LIME_DEST = ROOT + "model_outputs/lime/"
SHAP_DEST = ROOT + "model_outputs/shap/"
TREESHAP_DEST = ROOT + "model_outputs/treeshap/"


In [31]:
# # Define paths for testing locally  - comment out when not required 
# root = 'C:/Users/User/random_forest/'
# train_files = sorted(['train_fold_1_2017-01-06.csv'])
# test_files = ['test_fold_1_2017-01-06.csv']
# test_pred = ['test_prediction_fold_1_2017-01-06.csv']
# models = sorted(["random_forest_fold_1_2017-01-06.pkl"])
# LIME_DEST = root + "xai_test_results/lime/"
# SHAP_DEST = root + "xai_test_results/kernelshap/"
# TREESHAP_DEST = root + "xai_test_results/treeshap/"
# OUTPUT_DEST = root + "xai_test_results/"

In [32]:
# Required helper functions

def load_model(model_file_path):
    return pickle.load(open(model_file_path, 'rb'))

def save_json(dict_obj: Union[dict, list], path: str):
    writable_json = json.dumps(dict_obj, indent=4)
    with open(path, 'w') as file:
        file.write(writable_json)

Functions for LIME, TreeSHAP and KernelSHAP

In [33]:

# Functions used to generate LIME explanation and to save the explanation

# Function to convert the features in a LIME explanation object to a list of tuples
def get_lime_feature_list(exp_object, feat_names):
    exp_list = exp_object.as_map().get(1)
    exp_list_with_feature_names = [(feat_names[x[0]], x[1]) for x in exp_list]
    print(f'exp_list_with_feature_names = {exp_list_with_feature_names}')
    return exp_list_with_feature_names

def get_lime_feature_list_encoded(exp_object):
    exp_list = exp_object.as_map().get(1)
    print(f'exp_list = {exp_list}')
    return exp_list

def save_lime_explanation(exp, instance_loc, model_name, position, project_id,fold):

    # exp.show_in_notebook(show_table=True)
    # Save as html file
    filepath = f"{LIME_DEST}{fold}/{position}"
    try:
        os.makedirs(filepath, exist_ok = True)
        print("Directory '%s' created successfully" %filepath)
    except OSError as error:
        print("Directory '%s' can not be created")

    # Saving the explanation as an image
    exp.save_to_file(f'{LIME_DEST}{fold}/{position}/lime_exp_{project_id}_{model_name}.html')

    # Save as pyplot figure
    exp.as_pyplot_figure()
    plt.savefig(f'{LIME_DEST}{fold}/{position}/lime_exp_{project_id}_{model_name}.png')
    print(f"Saving lime exp for {project_id}_{model_name}")
    plt.show()
    plt.cla()

    return


# Function to generate explanations for a list of instances
def get_lime_for_list(instance_loc_list, x_test, explainer_lime, model, model_name, list_type, fold, num_of_feat):

    # Dictionary used to save the explanation objects
    exp_objects = {}
    exp_as_list_objects = {}
    
    for instance_loc in instance_loc_list:
        # Select instance
        instance = x_test.iloc[instance_loc]
        # Find its Project ID
        project_id = instance["Project ID"]
        # Drop the Project ID value from the instance since its not a feature
        instance = instance.drop(["Project ID"])
        # Get the explanation
        exp = explainer_lime.explain_instance(
            data_row=instance,
            predict_fn=model.predict_proba,
            num_features=num_of_feat
        )
        # Append exp object to dictionary
        exp_objects[project_id] = exp
        # Find the feature names as a list
        feat_names = instance.keys().to_list()
        # Get the explanation as a list of tuples and save
        exp_as_list = get_lime_feature_list(exp, feat_names)
        exp_as_list_objects[project_id] = exp_as_list
        # Save the explanation as a figure
        save_lime_explanation(exp, instance_loc, model_name, list_type, project_id, fold)
        
    return exp_objects, exp_as_list_objects

def get_lime_explanation(x_train, x_test, top_instance_loc_list, bottom_instance_loc_list, class_names, mode, model, model_name, fold, num_of_feat):

    # take the list of instances and save the explaination of each instance.
    # LIME: define the explainer
    # Ex: mode = 'classification' or 'regression'
    #     class_names = ['0', '1']

    categorical_feature_names = x_train.dtypes[x_train.dtypes==bool].index.to_list()
    categorical_feature_index = [x_train.columns.get_loc(col) for col in categorical_feature_names]
    
    # Define the explainer
    explainer_lime = lime_tabular.LimeTabularExplainer(
        training_data=np.array(x_train),
        feature_names=x_train.columns,
        categorical_features = categorical_feature_index,
        class_names=class_names,
        mode=mode
    )
    
    # Get LIME explanations for both top and bottom lists with Project ID
    print("Top list")
    exp_objects_top, exp_as_list_objects_top = get_lime_for_list(top_instance_loc_list, x_test, explainer_lime, model, model_name, "top", fold, num_of_feat)
    print("Bottom list")
    exp_objects_bottom, exp_as_list_objects_bottom = get_lime_for_list(bottom_instance_loc_list, x_test, explainer_lime, model, model_name, "bottom", fold, num_of_feat)
    
    return exp_as_list_objects_top, exp_as_list_objects_bottom




In [34]:
def save_treeshap_explanation(explainer_tree, exp, instance, model_name, list_type, project_id, fold):

    # Visualize and save
    filepath = f'{TREESHAP_DEST}{fold}/{list_type}/treeshap_exp_{project_id}_{model_name}.png'
    print(f"Saving treeshap exp for {filepath}")
    shap.force_plot(explainer_tree.expected_value[1], 
                exp[1],
                instance,
                show=False, 
                matplotlib=True, 
                text_rotation=45).savefig(filepath, format = "png", dpi = 150, bbox_inches = 'tight')
    return


def get_treeshap_feature_list(exp, feat_names, num_of_feat):
    
    # Create list of tuples
    shap_values = dict(zip(feat_names, exp[1][0]))
    sorted_shap_exp = sorted(shap_values.items(), key=lambda x:abs(x[1]), reverse=True)[:num_of_feat]

    #exp_list = exp[1].tolist()
    #exp_list_with_feature_names = [(feat_names[x], exp_list[x]) for x in range(len(exp_list))]
    #print(f'exp_list_with_feature_names = {exp_list_with_feature_names}')
    
    return sorted_shap_exp


# Function to generate explanations for a list of instances
def get_treeshap_for_list(instance_loc_list, x_test, explainer_tree, model_name, list_type, fold, num_of_feat):

    # Dictionary used to save the explanation objects
    exp_objects = {}
    exp_as_list_objects = {}
    
    for i, instance_loc in enumerate(instance_loc_list):
        # Select instance
        instance = x_test.iloc[[instance_loc]]
        # Find its Project ID
        project_id = instance["Project ID"].values[0]
        # Drop the Project ID value from the instance since its not a feature
        instance = instance.drop(["Project ID"], axis=1)
        # Get the explanation
        exp = explainer_tree.shap_values(instance)
        #print(f'exp[1] = {exp[1]}')
        # Append exp object to dictionary
        exp_objects[project_id] = exp
        # Find the feature names as a list
        feat_names = instance.columns.to_list()
        print(f'len(exp[1][0]) = {len(exp[1][0])}')
        print(f'len(feat_names) = {len(feat_names)}')
        print(f'exp[1][0] = {exp[1][0]}')
        print(f'feat_names top {num_of_feat} = {feat_names[:num_of_feat]}')
        
        # Get the explanation as a list of tuples and save
        exp_as_list = get_treeshap_feature_list(exp, feat_names, num_of_feat)
        exp_as_list_objects[project_id] = exp_as_list
        #print(exp_as_list)
        print(f'type(exp_as_list) = {type(exp_as_list)}')
        # Save the explanation as a figure
        save_treeshap_explanation(explainer_tree, exp, instance, model_name, list_type, project_id, fold)
        
    return exp_objects, exp_as_list_objects


def get_treeshap_explanation(x_train, x_test, top_instance_loc_list, bottom_instance_loc_list, model, model_name, fold, num_of_feat):
    
#     print(x_train.head())
    # Define the KernelSHAP explainer
#     explainer_tree = shap.TreeExplainer(model=model, data=x_train, model_output="raw")
    print(f"Treeshap explainer: Start training")
    explainer_tree = shap.TreeExplainer(model=model, feature_perturbation='interventional', data=x_train, model_output="raw")
    print(f"Treeshap explainer: Done training")
    
    filepath = f'{TREESHAP_DEST}{fold}/top/'
    try:
        os.makedirs(filepath, exist_ok = True)
        print("Directory '%s' created successfully" %filepath)
    except OSError as error:
        print("Directory '%s' can not be created")
        
    filepath = f'{TREESHAP_DEST}{fold}/bottom/'
    try:
        os.makedirs(filepath, exist_ok = True)
        print("Directory '%s' created successfully" %filepath)
    except OSError as error:
        print("Directory '%s' can not be created")
            
    print("Top list")          
    exp_objects_top, exp_as_list_objects_top = get_treeshap_for_list(top_instance_loc_list, x_test, explainer_tree, model_name, "top", fold, num_of_feat)
    
    print("Bottom list")
    exp_objects_bottom, exp_as_list_objects_bottom = get_treeshap_for_list(bottom_instance_loc_list, x_test, explainer_tree, model_name, "bottom", fold, num_of_feat)
    
    return exp_as_list_objects_top, exp_as_list_objects_bottom



In [35]:
def save_kernelshap_explanation(explainer_tree, exp, instance, model_name, list_type, project_id, fold):

    # Visualize and save
    filepath = f'{SHAP_DEST}{fold}/{list_type}/kernelshap_exp_{project_id}_{model_name}.png'
    print(f"Saving kernelshap exp for {filepath}")
    shap.force_plot(explainer_tree.expected_value[1], 
                exp[1],
                instance,
                show=False, 
                matplotlib=True, 
                text_rotation=45).savefig(filepath, format = "png", dpi = 150, bbox_inches = 'tight')
    return

def get_kernelshap_feature_list(exp, feat_names, num_of_feat):
    
    # Create list of tuples
    shap_values = dict(zip(feat_names, exp[1]))
    sorted_shap_exp = sorted(shap_values.items(), key=lambda x:abs(x[1]), reverse=True)[:num_of_feat]

    #exp_list = exp[1].tolist()
    #exp_list_with_feature_names = [(feat_names[x], exp_list[x]) for x in range(len(exp_list))]
    #print(f'exp_list_with_feature_names = {exp_list_with_feature_names}')
    
    return sorted_shap_exp


# Function to generate explanations for a list of instances
def get_kernelshap_for_list(instance_loc_list, x_test, explainer_tree, model_name, list_type, fold, num_of_feat, nsamples):

    # Dictionary used to save the explanation objects
    exp_objects = {}
    exp_as_list_objects = {}
    
    for i, instance_loc in enumerate(instance_loc_list):
        # Select instance
        instance = x_test.iloc[instance_loc]
        # Find its Project ID
        project_id = instance["Project ID"]
        # Drop the Project ID value from the instance since its not a feature
        instance = instance.drop(["Project ID"])
        
        # Get the explanation
        exp = explainer_tree.shap_values(instance, nsamples=nsamples) # nsamples can be either 'auto' or an int
        print(f'len(exp[1]) = {len(exp[1])}')
        print(f'exp[1] = {exp[1]}')
        # Find the feature names as a list
        feat_names = instance.keys().to_list()
        print(f'len(feat_names) = {len(feat_names)}')
        print(f'feat_names top entries = {feat_names[:num_of_feat]}')
        
        # Append exp object to dictionary
        exp_objects[project_id] = exp
        # Get the explanation as a list of tuples and save
        exp_as_list = get_kernelshap_feature_list(exp, feat_names, num_of_feat)
        exp_as_list_objects[project_id] = exp_as_list
        
        print(f'type(exp_as_list) = {type(exp_as_list)}')
        # Save the explanation as a figure
        save_kernelshap_explanation(explainer_tree, exp, instance, model_name, list_type, project_id, fold)
        
    return exp_objects, exp_as_list_objects



def get_kernelshap_explanation(x_train, x_test, top_instance_loc_list, bottom_instance_loc_list, model, model_name, fold, num_of_feat, nsamples):
    
    # Define the KernelSHAP explainer
    print("Kernel Explainer Loading ..... ")
    explainer_shap = shap.KernelExplainer(model=model.predict_proba, data=x_train)
    print("Kernel Explainer : Done training")
    
    filepath = f'{SHAP_DEST}{fold}/top/'
    
    try:
        os.makedirs(filepath, exist_ok = True)
        print("Directory '%s' created successfully" %filepath)
    except OSError as error:
        print("Directory '%s' can not be created")
        
    filepath = f'{SHAP_DEST}{fold}/bottom/'
    
    try:
        os.makedirs(filepath, exist_ok = True)
        print("Directory '%s' created successfully" %filepath)
    except OSError as error:
        print("Directory '%s' can not be created")
            
            
    print("Top list")    
    exp_objects_top, exp_as_list_objects_top = get_kernelshap_for_list(top_instance_loc_list, x_test, explainer_shap, model_name, "top", fold, num_of_feat, nsamples)   
    print("Bottom list")
    exp_objects_bottom, exp_as_list_objects_bottom = get_kernelshap_for_list(bottom_instance_loc_list, x_test, explainer_shap, model_name, "bottom", fold, num_of_feat, nsamples)

    return exp_as_list_objects_top, exp_as_list_objects_bottom

In [36]:
# Function used to create csv using explanations
def create_csv_for_exps(feat_names, lime_dict, treeshap_dict, kernelshap_dict, list_type, fold):
    # First create an empty df
    data = pd.DataFrame()
    # Create feature column
    data['features'] = feat_names
    # Create columns to store lime, treeshap and kernalshap values for each project in the dictionary
    projects_list = list(lime_dict.keys())
    print(f'Projects list = {projects_list}')
    for project in projects_list:
        # Create empty columns
        data[f'lime_{list_type}_{project}'] = "" 
        data[f'treeshap_{list_type}_{project}'] = ""
        data[f'kernelshap_{list_type}_{project}'] = ""

        # Get lime, treeshap and kernelshap lists of tuples
        lime_list = lime_dict[project]
        treeshap_list = treeshap_dict[project]
        kernelshap_list = kernelshap_dict[project]

        # Fill the necessary cells with data
        # For lime
        for item in lime_list:
            feature = item[0]
            value = item[1]
            data.loc[data['features'] == feature, f'lime_{list_type}_{project}'] = value
        # For treeshap
        for item in treeshap_list:
            feature = item[0]
            value = item[1]
            data.loc[data['features'] == feature, f'treeshap_{list_type}_{project}'] = value
        # For kernelshap
        for item in kernelshap_list:
            feature = item[0]
            value = item[1]
            data.loc[data['features'] == feature, f'kernelshap_{list_type}_{project}'] = value

    
    print(data.columns)
    # Save as csv
    data.to_csv(OUTPUT_DEST+f'all_explanations_{fold}_{list_type}.csv') #### MAKE SURE OUTPUT_DEST IS DEFINED
    
    return data

In [37]:
# New pipeline for running the explainations - call this from main
def explanations_pipeline(root, model_paths, train_paths, test_paths, pred_paths, model_name):
  """this pipeline will generate explanations for the given model paths

  Args:
      root (_type_): This is the root path of the model artifacts
      model_paths (_type_): these are the paths to the model artifacts
      train_paths (_type_): these are the paths to the train artifacts
      test_paths (_type_): these are the paths to the test artifacts
      pred_paths (_type_): these are the paths to the prediction artifacts
      model_name (str): this is the model name as a string for identification. ex: "random forest"
      
      all the artifacts should be a list of files and should be in the same order
  """
  assert len(train_paths) == len(test_paths), "There should be same number of train paths and test paths"
  assert len(test_paths) == len(pred_paths), "There should be same number of predictions paths and test paths"
  assert len(model_paths) == len(test_paths), "There should be same number of model_paths paths and test paths"

  num_of_explanations = 1 # Should be 50 #### CHANGE THE VALUE TO 50
  num_of_records = 20 # Should be 1000 #### CHANGE THE VALUE TO 1000
  rows = 1000 # Should be None #### CHANGE THE VALUE TO NONE

  # Dictionary to store all explanations
  all_exp_dict = {}

  # Loop to generate explanations for samples from each fold
  for i in range(len(train_paths)):
    print(f"Fold {i} .............................")
    x_train = pd.read_csv(os.path.join(root, train_paths[i]), nrows=rows)
    x_train_cleaned = x_train.drop(["Unnamed: 0", "Project ID", "Label"], axis=1)

    fold1 = pd.read_csv(os.path.join(root,test_paths[i]), nrows=rows)
    fold_pred =  pd.read_csv(os.path.join(root,  pred_paths[i]), nrows=rows)
    Fold1 = pd.concat([fold1, fold_pred["1"]],axis=1)
    Fold1 = Fold1.drop(["Unnamed: 0"],axis=1)
    Fold1_sort = Fold1.sort_values(["1"], ascending=False)
    #Fold1_sort.head()
    x_test_with_id = Fold1_sort.drop([ "Label", "1"],axis=1)

    # Select n samples each from top and bottom k records
    print(f"Sampling the top {num_of_explanations} and bottom {num_of_explanations}")
    top_instance_loc_list = random.sample(range(num_of_records), num_of_explanations)
    bottom_instance_loc_list = random.sample(range(x_test_with_id.shape[0]-num_of_records , x_test_with_id.shape[0]), num_of_explanations)
    print(f'top_instance_loc_list = {top_instance_loc_list}')
    print(f'bottom_instance_loc_list = {bottom_instance_loc_list}')
      
    # Define the number of features required to display after generating the explanations
    #num_of_feat = x_test_with_id.shape[1] - 1
    num_of_feat = 10 #### COMMENT THIS UNCOMMENT THE PREVIOUS LINE
    x_train_n_rows = 100 #### CHANGE IF NECESSARY

    print(f"Model {model_paths} is loading")

    # Load the saved model
    model = load_model(os.path.join(root, model_paths[i]))

    # Get explanations
    print(f"Explanation for Lime")
    lime_list_objects_top, lime_list_objects_bottom = get_lime_explanation(x_train_cleaned[:x_train_n_rows].astype("float64"), x_test_with_id, top_instance_loc_list, bottom_instance_loc_list, ["0", "1"], "classification", model, model_name, f"Fold{i}", num_of_feat)
    print(f"Explanation for Kernel Shap" )
    kernelshap_list_objects_top, kernelshap_list_objects_bottom = get_kernelshap_explanation(x_train_cleaned[:x_train_n_rows].astype("float64"), x_test_with_id, top_instance_loc_list, bottom_instance_loc_list, model, model_name, f"Fold{i}", num_of_feat, 100) 
    print(f"Explanation for Tree Shap")
    treeshap_list_objects_top, treeshap_list_objects_bottom = get_treeshap_explanation(x_train_cleaned[:x_train_n_rows].astype("float64"), x_test_with_id, top_instance_loc_list, bottom_instance_loc_list, model, model_name, f"Fold{i}", num_of_feat) 

    #print(lime_list_objects_top)
    #print(treeshap_list_objects_top)
    #print(kernelshap_list_objects_top)

    # Create dictionaries to store the explanations as json
    current_fold_dict = {
        'lime': {
            'top': lime_list_objects_top,
            'bottom': lime_list_objects_bottom
        },
        'treeshap': {
            'top': treeshap_list_objects_top,
            'bottom': treeshap_list_objects_bottom
        },
        'kernelshap': {
            'top': kernelshap_list_objects_top,
            'bottom': kernelshap_list_objects_bottom
        }
    }
    all_exp_dict[f"fold{i}"] = current_fold_dict

  # Get a list of all the features
  list_of_features = x_test_with_id.drop(["Project ID"],axis=1).columns.to_list()
  # Storing explanations as json
  save_json(all_exp_dict, OUTPUT_DEST+'all_exp.json') #### MAKE SURE OUTPUT_DEST IS DEFINED

  # Creating top and bottom csvs for each fold to summarize explanations
  for i in range(len(train_paths)):
      # For the top list
      top_df = create_csv_for_exps(list_of_features, all_exp_dict[f"fold{i}"]["lime"]["top"], all_exp_dict[f"fold{i}"]["treeshap"]["top"], all_exp_dict[f"fold{i}"]["kernelshap"]["top"], "top", f"fold{i}")
      # For the bottom list
      bottom_df = create_csv_for_exps(list_of_features, all_exp_dict[f"fold{i}"]["lime"]["bottom"], all_exp_dict[f"fold{i}"]["treeshap"]["bottom"], all_exp_dict[f"fold{i}"]["kernelshap"]["bottom"], "bottom", f"fold{i}")
    
  
  return

In [40]:
# New pipeline for running the explainations - call this from main
def explanations_pipeline_without_from_saving_files( root_test, model_paths, data_folds,  pred_paths, model_name):
  """this pipeline will generate explanations for the given model paths

  Args:
      root (_type_): This is the root path of the model artifacts
      model_paths (_type_): these are the paths to the model artifacts
      train_paths (_type_): these are the paths to the train artifacts
      test_paths (_type_): these are the paths to the test artifacts
      pred_paths (_type_): these are the paths to the prediction artifacts
      model_name (str): this is the model name as a string for identification. ex: "random forest"
      
      all the artifacts should be a list of files and should be in the same order
  """
  # assert len(train_paths) == len(test_paths), "There should be same number of train paths and test paths"
  # assert len(test_paths) == len(pred_paths), "There should be same number of predictions paths and test paths"
  # assert len(model_paths) == len(test_paths), "There should be same number of model_paths paths and test paths"

  num_of_explanations = 1 # Should be 50 #### CHANGE THE VALUE TO 50
  num_of_records = 20 # Should be 1000 #### CHANGE THE VALUE TO 1000
  rows = 100 # Should be None #### CHANGE THE VALUE TO NONE

  # Dictionary to store all explanations
  all_exp_dict = {}

  # Loop to generate explanations for samples from each fold
  for i in range(len(data_folds)-1, 0, -1):
    print(i)
    print(f"Fold {i} .............................")

    print("Data Fold", data_folds[i])
      
    if rows:
        x_train = data_folds[i]["x_train"][:rows]
    else:
        x_train = data_folds[i]["x_train"]
    x_train =  x_train.dropna()
    x_train_cleaned = x_train.drop(["Unnamed: 0", "Project ID"], axis=1)

    if rows:
        fold1 = data_folds[i]["x_test"][:rows]
    else:
        fold1 = data_folds[i]["x_test"]

    # fold1 = pd.read_csv(os.path.join(root,test_paths[i]), nrows=rows)
    fold_pred =  pd.read_csv(os.path.join(root_test,  pred_paths[i]), nrows=rows)
    Fold1 = pd.concat([fold1, fold_pred["1"]],axis=1)
    Fold1 = Fold1.drop(["Unnamed: 0"],axis=1)
    Fold1_sort = Fold1.sort_values(["1"], ascending=False)
    x_test_with_id = Fold1_sort.drop(["1"],axis=1)
    print(x_test_with_id)
    # exit(0)

    # Select n samples each from top and bottom k records
    print(f"Sampling the top {num_of_explanations} and bottom {num_of_explanations}")
    top_instance_loc_list = random.sample(range(num_of_records), num_of_explanations)
    bottom_instance_loc_list = random.sample(range(x_test_with_id.shape[0]-num_of_records , x_test_with_id.shape[0]), num_of_explanations)
    print(f'top_instance_loc_list = {top_instance_loc_list}')
    print(f'bottom_instance_loc_list = {bottom_instance_loc_list}')
      
    # Define the number of features required to display after generating the explanations
    #num_of_feat = x_test_with_id.shape[1] - 1
    num_of_feat = 10 #### COMMENT THIS UNCOMMENT THE PREVIOUS LINE
    x_train_n_rows = 100 #### CHANGE IF NECESSARY

    print(f"Model {model_paths} is loading")
      
    print(os.path.join(root_test, model_paths[i]))

    # Load the saved model
    model = load_model(os.path.join(root_test, model_paths[i]))


    # Get explanations
    print(f"Explanation for Lime")
    lime_list_objects_top, lime_list_objects_bottom = get_lime_explanation(x_train_cleaned[:x_train_n_rows].astype("float64"), x_test_with_id, top_instance_loc_list, bottom_instance_loc_list, ["0", "1"], "classification", model, model_name, f"Fold{i}", num_of_feat)
    print(f"Explanation for Kernel Shap" )
    kernelshap_list_objects_top, kernelshap_list_objects_bottom = get_kernelshap_explanation(x_train_cleaned[:x_train_n_rows].astype("float64"), x_test_with_id, top_instance_loc_list, bottom_instance_loc_list, model, model_name, f"Fold{i}", num_of_feat, 100) 
    print(f"Explanation for Tree Shap")
    treeshap_list_objects_top, treeshap_list_objects_bottom = get_treeshap_explanation(x_train_cleaned[:x_train_n_rows].astype("float64"), x_test_with_id, top_instance_loc_list, bottom_instance_loc_list, model, model_name, f"Fold{i}", num_of_feat) 

    #print(lime_list_objects_top)
    #print(treeshap_list_objects_top)
    #print(kernelshap_list_objects_top)

    # Create dictionaries to store the explanations as json
    current_fold_dict = {
        'lime': {
            'top': lime_list_objects_top,
            'bottom': lime_list_objects_bottom
        },
        'treeshap': {
            'top': treeshap_list_objects_top,
            'bottom': treeshap_list_objects_bottom
        },
        'kernelshap': {
            'top': kernelshap_list_objects_top,
            'bottom': kernelshap_list_objects_bottom
        }
    }
    all_exp_dict[f"fold{i}"] = current_fold_dict
    # return

  

  # Get a list of all the features
  list_of_features = x_test_with_id.drop(["Project ID"],axis=1).columns.to_list()
  # Storing explanations as json
  save_json(all_exp_dict, OUTPUT_DEST+'all_exp.json') #### MAKE SURE OUTPUT_DEST IS DEFINED

  # Creating top and bottom csvs for each fold to summarize explanations
  for i in range(len(train_paths)):
      # For the top list
      top_df = create_csv_for_exps(list_of_features, all_exp_dict[f"fold{i}"]["lime"]["top"], all_exp_dict[f"fold{i}"]["treeshap"]["top"], all_exp_dict[f"fold{i}"]["kernelshap"]["top"], "top", f"fold{i}")
      # For the bottom list
      bottom_df = create_csv_for_exps(list_of_features, all_exp_dict[f"fold{i}"]["lime"]["bottom"], all_exp_dict[f"fold{i}"]["treeshap"]["bottom"], all_exp_dict[f"fold{i}"]["kernelshap"]["bottom"], "bottom", f"fold{i}")
  # exit(0)
  
  return

In [41]:
explanations_pipeline_without_from_saving_files(root_test, models, data_folds, test_pred, "random_forest")

5
Fold 5 .............................
Data Fold {'fold_no': 6, 'start_date': Timestamp('2014-05-17 00:00:00'), 'x_train':         Unnamed: 0                        Project ID  Project Cost  \
10              15  c19a46b078ca90fe7c7fc6de086703ee        673.81   
25              38  8887c91f37a4bdb024421aae7dd6e474        889.34   
30              52  82c9238ee69a98ab4aed112d2f65086b       1199.49   
33              57  e478c8648f403ed46bdfb9f4e58aed2e       1581.14   
38              64  9d841c104513a7dd9c54a62dd9e81013        519.00   
...            ...                               ...           ...   
578029     1109869  a6fb108a71f8e44a37acb69d13f3e60f        393.46   
578040     1109898  ef65189660fa2a3e4a7ca9760c077537       1570.91   
578052     1109923  60e209321380a60a2270682561d65ae1        439.40   
578053     1109925  7cb11121a5c371dd1ca0eda5c1c179fc        823.65   
578069     1109946  51e2eeb4af5a360d2a2f1a076ee4342a        743.95   

        School Percentage Free Lunch

X does not have valid feature names, but RandomForestClassifier was fitted with feature names


ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [43]:
# explanations_pipeline(root, models, train_files, test_files, test_pred, "random_forest")